<a href="https://colab.research.google.com/github/alheliou/Bias_mitigation/blob/main/UPP26/TD5_inprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TD 5: Mitigation des biais avec une méthode de in-processsing Prejudice Remover

The aim of this notebook is to use the Prejudice Remover in-processing approach and analyse its impact on the model output.
In terms of Machine Learning we will go a bit further in the train/valid/test paradigm.

The model has to be learn on the train dataset, then the model parameters has to be optimized on the valid dataset, and finally the model performance is evaluated on the test dataset.
No choice/decision etc can be taken depending on the test dataset. This could result on an overfitting on the test dataset.

Here you will manipulate:
- Prejudice Remover approach as a black box
- Training of the prejudice remover using the train/valid paradigm. to choice the 'best' threshold
- Combine Prejudice Remover with Reweighing

As a reminder of pre-processing approach we encourage you to :
- analyse the impact of the Reweighing on different model (Logistic Regression, Decision Tree, Random Forest, etc.)


## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings
  The next cell of code are to execute only once per colab environment


#### Python env creation

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```
### Local Settings

#### 1. Uv installation


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`


#### 3. Python env creation

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```


## 1. Import and load the dataset

In [ ]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", append=True, category=UserWarning)
# Datasets
from aif360.datasets import MEPSDataset19

# Fairness metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

MEPSDataset19_data = MEPSDataset19()
(dataset_orig_panel19_train, dataset_orig_panel19_val, dataset_orig_panel19_test) = (
    MEPSDataset19().split([0.5, 0.8], shuffle=True)
)

In [ ]:
len(dataset_orig_panel19_train.instance_weights), len(
    dataset_orig_panel19_val.instance_weights
), len(dataset_orig_panel19_test.instance_weights)

In [ ]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights

In [ ]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

In [ ]:
from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score


# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

## Learning a Prejudice Remover model on the training dataset, and choose the best parameters with the validation dataset

In [ ]:
# Bias mitigation techniques
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover

### Question1 : Learn a Standard Scaler on the training dataset features, its output will be used as input of the model learned

In [ ]:
print("TODO")

### Question2: Create a method to learn a Prejudice Remover on the train dataset and retrieve the model learned
Execute the method with the parameter eta arbitrarily set at 25.0



In [ ]:
print("TODO")

Le score du Prejudice Remover donne un sortie pour chaque instance une seule valeur, c'est un seuil, arbritrairement fixé à 0.5 par défault, qui permet à partir de ce score de décider la prédiction 1 ou 0.
Si le score est supérieur au seuil la prédiction est 1, sinon c'est 0.

### Validating: Choose the best parameters

Here there are two parameters :
- eta: fairness penalty parameter of the PR model
- thershold: the threshold of the binary classification

The threshold is used to obtains predictions from the model output.
The eta is used during the training

Question3: Create a method that will loop over 50 threshold ]0:0.5( and 5 values of ETA [1.0: 100.0], and outputs the metrics

In [ ]:
print("TODO")

### Question4 : Make plot to choose the best set of parameters

In [ ]:
print("TODO")

### Question 5: Evaluate : compute the metrics on the test dataset using the model learnt with the selected parameters


In [ ]:
print("TODO")

## Combine pre-processing and in-processing
### Question6: Redo the Prejudice Remover approach using first the Reweighing pre-processing

In [ ]:
print("TODO")

## Adversarial Debiasing

Adversarial debiasing [1] is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary's ability to determine the protected attribute from the predictions.

See [AIF360 tuto](https://github.com/Trusted-AI/AIF360/blob/main/examples/demo_adversarial_debiasing.ipynb)

Here we show how to learn and Adversarial Debiasing with the argumetn debias set to False

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

sess = tf.Session()

plain_model = AdversarialDebiasing(
    unprivileged_groups=[{'RACE': 0.0}],
    privileged_groups=[{'RACE': 1.0}],
    scope_name='plain_classifier',
    debias=False,
    sess=sess)

plain_model.fit(dataset_orig_panel19_train)

In [ ]:
# Apply the plain model to train and val data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_panel19_train)
dataset_nodebiasing_val = plain_model.predict(dataset_orig_panel19_val)

In [ ]:
get_metrics(
    y_true = dataset_orig_panel19_train.labels[:,0],
    y_pred= dataset_nodebiasing_train.labels[:,0],
    prot_attr= dataset_orig_panel19_train.protected_attributes[:,0],
    sample_weight= dataset_orig_panel19_train.instance_weights
)

In [ ]:
get_metrics(
    y_true = dataset_orig_panel19_val.labels[:,0],
    y_pred= dataset_nodebiasing_val.labels[:,0],
    prot_attr= dataset_orig_panel19_val.protected_attributes[:,0],
    sample_weight= dataset_orig_panel19_val.instance_weights
)

In [ ]:
sess.close()
tf.reset_default_graph()


### Question 7: Redo the same (learn and Adversarial Debiasing) with the argument debias set to True

Compare the metrics outputed

In [ ]:
print("TODO")

### Question 8: Combine the Reweighing with the Adversarial Debiasing

In [ ]:
print("TODO")

This in-processing approach does not seem compatible withe the Reweighing, has the df_bias_amplification is high and the disparate impact ratio is not improved by the use of the reweighing has pre-processing.
Although very efficient on the fairness metrics of the dataset, the Reweighing is not convenient for every kind of machine learning algo.



## Analysis of the influence of Reweighing

### QUESTION 9 : Pour aller plus loin, étudier l'impact du Reweighing sur différents modèles notamment les arbres de décision

In [ ]:
print("TODO")